# Car Detection using OpenCV

This code performs car detection using OpenCV. It allows you to either detect cars in real-time using a camera or analyze a video file.

## Functionality

1. **Input Source Selection**: The code prompts the user to enter the source type: `'camera'` for real-time camera detection or `'video'` to analyze a video file.
   - If `'camera'` is selected, it captures video frames from the default camera (index 0).
   - If `'video'` is selected, the user is asked to provide the path to the video file.

2. **Video Capture**: The code initializes a `cv2.VideoCapture` object to read frames from the selected source.
   - If the source is a camera, it opens the default camera.
   - If the source is a video file, it opens the specified file.

3. **Background Subtraction**: The code creates a background subtractor using `cv2.createBackgroundSubtractorMOG2`.
   - This subtractor is used to separate moving objects (such as cars) from the background.
   - The parameters `history`, `varThreshold`, and `detectShadows` control the behavior of the background subtractor.

4. **Detection Loop**: The code enters a loop to process each frame from the video source.
   - It reads the next frame using `cap.read()`.
   - The background subtractor is applied to the frame, generating a binary mask that highlights moving objects.
   - A binary threshold is applied to the mask to obtain a more distinct separation between the objects and the background.

5. **Contour Detection**: The code uses `cv2.findContours` to detect contours in the binary mask.
   - Contours are continuous curves that form the boundaries of the objects in the mask.
   - The `cv2.RETR_TREE` retrieval mode and `cv2.CHAIN_APPROX_SIMPLE` contour approximation method are used.

6. **Car Detection**: The code iterates over the detected contours and applies a size threshold to identify car-like objects.
   - If the contour area is greater than or equal to 5000 pixels, it is considered a car.
   - A bounding rectangle is drawn around each detected car using `cv2.rectangle`.

7. **Display**: The code displays the processed frame with the bounding rectangles using `cv2.imshow`.

8. **Termination**: The code waits for a key press using `cv2.waitKey`. If the 'Esc' key is pressed (key code 27), the loop is terminated.

9. **Cleanup**: After the loop, the video source is released using `cap.release()`, and all windows are closed using `cv2.destroyAllWindows()`.

## Usage

1. Run the code in a Python environment that has OpenCV installed.

2. Follow the prompts to select the source type and, if applicable, enter the path to the video file.

3. The code will start processing the frames and display the output in a separate window.

4. To exit the program, press the 'Esc' key.

In [1]:
import cv2
from tracker import *

In [2]:
cap = cv2.VideoCapture('task1/test2.mp4')

In [3]:
car_tracker_ob = EuclideanDistTracker()
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=100)

In [4]:
while True:
    ret, frame = cap.read()
    roi = frame[200:-100, 300:-690]
   
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detected_cars = []
    for cnt in contours:
        # cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
        if cv2.contourArea(cnt) > 5000:
            x, y, w, h = cv2.boundingRect(cnt)
            detected_object = [x, y, w, h]
            if detected_object not in detected_cars:
                detected_cars.append([x, y, w, h])
    cars_tracked = car_tracker_ob.update(detected_cars)

    for car in cars_tracked:
        x, y, w, h, id = car
        cv2.putText(roi,str(id), (x, y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 3)


    # cv2.imshow('Frame', roi)
    cv2.imshow('Frame', frame)
    # cv2.imshow('Mask', mask)

    key = cv2.waitKey(30)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable